In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import cv2
import mediapipe as mp

# Step 1: Generate Synthetic Data (Replace with actual data loading)
num_samples = 1000
num_landmarks = 33
num_coordinates = 3
num_measurements = 5

landmarks = np.random.rand(num_samples, num_landmarks * num_coordinates)
measurements = np.random.rand(num_samples, num_measurements)

train_landmarks, test_landmarks, train_measurements, test_measurements = train_test_split(
    landmarks, measurements, test_size=0.2, random_state=42)

# Step 2: Build and Train the Model
model = models.Sequential([
    layers.Input(shape=(num_landmarks * num_coordinates,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_measurements)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_landmarks, train_measurements, epochs=50, validation_split=0.2)
test_loss = model.evaluate(test_landmarks, test_measurements)
print(f'Test Loss: {test_loss}')
model.save('body_measurement_model.h5')

# Step 3: Use the Pre-trained Model in the Application
model = tf.keras.models.load_model('body_measurement_model.h5')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture(0)

def process_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    if results.pose_landmarks:
        return results.pose_landmarks
    return None

def draw_landmarks(frame, pose_landmarks):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing.draw_landmarks(frame, pose_landmarks, mp_pose.POSE_CONNECTIONS)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    pose_landmarks = process_frame(frame)
    if pose_landmarks:
        draw_landmarks(frame, pose_landmarks)
        input_data = np.array([[landmark.x, landmark.y, landmark.z] for landmark in pose_landmarks.landmark]).flatten()
        input_data = np.expand_dims(input_data, axis=0)
        predictions = model.predict(input_data)
        print(f'Predicted Measurements: {predictions}')
    cv2.imshow('Camera Feed', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        breakii

cap.release()
cv2.destroyAllWindows()


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.2792 - val_loss: 0.1049
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1031 - val_loss: 0.0954
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0941 - val_loss: 0.0969
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0878 - val_loss: 0.0945
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0831 - val_loss: 0.0954
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0822 - val_loss: 0.0943
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0801 - val_loss: 0.0928
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0781 - val_loss: 0.0956
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0767 - val_loss: 0.0941
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0736 - val_loss: 0.0943
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0719 - val_loss: 0.0956
Epoch 12/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0708 - val_l

Test Loss: 0.1275629848241806
